This notebook works over the Excel spreadsheet of FWS ESA Work Plan Species to put the data into our working Biogeographic Information System database (currently a MongoDB database). That turns the records into a form that we can process through the Taxa Information Registry to assemble attributes for further analysis and reporting. The spreadseet in Excel format was put together by USGS Ecosystems personnel. FWS has this list on a [web site](https://www.fws.gov/endangered/what-we-do/listing-workplan.html) and in a [PDF document](https://www.fws.gov/endangered/esa-library/pdf/Listing%207-Year%20Workplan%20Sept%202016.pdf) with some additional information.

In [1]:
import pandas as pd
from bis2 import dd
from IPython.display import display
from datetime import datetime

A couple of new functions to move into the BIS somewhere.

In [2]:
def packageESASpeciesRow(row):
    submittedData = {}
    submittedData["Scientific Name"] = row["Scientific Name (Revised List)"]
    submittedData["Species Record Reference"] = row["ScientificNameLink"] 
    submittedData["Common Name"] = row["Species Name (Common)"]
    submittedData["Grouping"] = row["Grouping"]
    submittedData["Lead FWS Region"] = row["Lead FWS Regional Office"]
    submittedData["Species Range"] = row["Species Range"]
    return submittedData

def lookupState(stateAbbr):
    import us
    try:
        return {"name":us.states.mapping('abbr', 'name')[stateAbbr],"fips":us.states.mapping('abbr', 'fips')[stateAbbr]}
    except:
        return None

Read the spreadsheet from the Excel file into a Pandas dataframe. I extracted out the ECOS link from the hyperlink on scientific name into its own attribute but kept the spreadsheet intact otherwise.

In [3]:
df = pd.read_excel("FWS ESA Work Plan Species list for CSS.xlsx", sheet_name='Sheet1')
display (df)

,Grouping,Species Name (Common),Scientific Name (Revised List),Lead FWS Regional Office,Species Range,ScientificNameLink
0,Amphibians,streamside salamander,Ambystoma barbouri,R4,"AL, KY, OH, TN, WV",https://ecos.fws.gov/ecp/species/9776
1,Amphibians,Boreal toad (Eastern population),Anaxyrus boreas boreas,R6,"CO, ID, NM, NV, UT, WY",https://ecos.fws.gov/ecp/species/1114
2,Amphibians,Inyo Mountains slender salamander,Batrachoseps campi,R8,CA,https://ecos.fws.gov/ecp/species/2095
3,Amphibians,lesser slender salamander,Batrachoseps minor,R8,CA,https://ecos.fws.gov/ecp/species/9277
4,Amphibians,relictual slender salamander,Batrachoseps relictus,R8,CA,https://ecos.fws.gov/ecp/species/7408
5,Amphibians,Kern Plateau salamander,Batrachoseps robustus,R8,CA,https://ecos.fws.gov/ecp/species/9274
6,Amphibians,Kern Canyon slender salamander,Batrachoseps simatus,R8,CA,https://ecos.fws.gov/ecp/species/5736
7,Amphibians,Oregon slender salamander,Batrachoseps wrighti,R1,"OR, WA",https://ecos.fws.gov/ecp/species/913
8,Amphibians,Arizona toad,Bufo microscaphus microscaphus,R2,"AZ, CA, NM, NV, UT",https://ecos.fws.gov/ecp/species/2077
9,Amphibians,hellbender,Cryptobranchus alleganiensis,R3,"AL, AR, GA, IL, IN, KY, MD, MO, MS, NC, NY, OH...",https://ecos.fws.gov/ecp0/profile/speciesProfi...


Get the collection we want to put these data into from the Data Distillery MongoDB instance.

In [4]:
bisDB = dd.getDB("bis")
esaWPSpecies = bisDB["FWS ESA Work Plan Species"]

Insert the original submitted data into the MongoDB collection as a data structure.

In [5]:
fwsESASpeciesList = []

for index,row in df.iterrows():
    speciesRecord = {}
    speciesRecord["Submitted Data"] = packageESASpeciesRow(row)
    speciesRecord["Processing Metadata"] = {"Date Created from Source":datetime.utcnow().isoformat()}
    fwsESASpeciesList.append(speciesRecord)

esaWPSpecies.delete_many({})
esaWPSpecies.insert_many(fwsESASpeciesList)

Parse out the range places (state abbreviations and other place names) into their own data structure for later processing and comparison.

In [7]:
for record in esaWPSpecies.find():
    fwsRange = {"US States":[],"US State List":[],"Other Places":[]}
    for rangePlace in record["Submitted Data"]["Species Range"].replace(", ",",").split(","):
        rangeState = lookupState(rangePlace)
        if rangeState is None:
            fwsRange["Other Places"].append({rangePlace:{}})
        else:
            fwsRange["US States"].append({rangePlace:rangeState})
            fwsRange["US State List"].append(rangePlace)

    if len(fwsRange["Other Places"]) == 0:
        del fwsRange["Other Places"]

    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"FWS Range":fwsRange}})


Show what the final data structure looks like at this point.

In [8]:
display(esaWPSpecies.find_one())

{'FWS Range': {'US State List': ['AL', 'KY', 'OH', 'TN', 'WV'],
  'US States': [{'AL': {'fips': '01', 'name': 'Alabama'}},
   {'KY': {'fips': '21', 'name': 'Kentucky'}},
   {'OH': {'fips': '39', 'name': 'Ohio'}},
   {'TN': {'fips': '47', 'name': 'Tennessee'}},
   {'WV': {'fips': '54', 'name': 'West Virginia'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-02-21T13:38:13.076667'},
 'Submitted Data': {'Common Name': 'streamside salamander',
  'Grouping': 'Amphibians',
  'Lead FWS Region': 'R4',
  'Scientific Name': 'Ambystoma barbouri',
  'Species Range': 'AL, KY, OH, TN, WV',
  'Species Record Reference': 'https://ecos.fws.gov/ecp/species/9776'},
 '_id': ObjectId('5a8d76450601ba07d55ba317')}